# Zaawansowany Python, lab 3

## 1. Metoda klasy oraz metoda statyczna

**Listing 1**

In [2]:
from enum import Enum
import inspect


class FieldType(Enum):
    INTEGER = 1
    FLOAT = 2
    STRING = 3 
    DATE = 4


class Field:

    def __init__(self, field_type: FieldType):
        self.field_type = field_type
    
    def get_fieldtype(self):
        return self.field_type

    def __str__(self):
        return self.field_type.__class__.__name__


class Model:
    
    def __init__(self, db_table=None):
        if db_table is None:
            self.db_table = f'db_{self.__class__.__name__.lower()}'
        else:
            self.db_table = db_table

    def get_fields(self):
        fields = {}
        for name, obj in inspect.getmembers(self):
            if isinstance(obj, Field):
                fields[name] = obj.get_fieldtype()
        return fields

    def __setattr__(self, attr, val):
        for name, obj in inspect.getmembers(self):
            if name == attr and isinstance(obj, Field):
                obj.value = val
                return
        super().__setattr__(attr, val)

    @staticmethod
    def generate_table_for_name(name: str):
        """ metoda statyczna wzracająca nazwę tabeli dla przykładowej nazwy modelu """
        return f'db_{name.lower()}'
        
    @classmethod
    def from_dict(cls, name: str, fields: dict[str, Field]):
        # tu wykorzystać match case z mapowaniem słownika
        for field in fields.items():
            
            match field:
                case (str(), Field()):
                    setattr(cls, field[0], field[1])
        
        model = cls()
        model.db_table = f'db_{name.lower()}'
        return model
        

In [34]:
# wykorzystanie klasy typu Enum
print(FieldType.INTEGER)
print(FieldType.INTEGER.name)
print(FieldType.INTEGER.value)
print(FieldType)

FieldType.INTEGER
INTEGER
1
<enum 'FieldType'>


In [35]:
# wywołanie metody statycznej
Model.generate_table_for_name('Osoba')

'db_osoba'

In [62]:
# czy można stworzyć instancję klasy Model?
model = Model()
# Można. Klasa nie jest abstrakcyjna

In [54]:
model.get_fields()

{}

In [55]:
model.id = Field(FieldType.INTEGER)

In [56]:
model.get_fields()

{'id': <FieldType.INTEGER: 1>}

In [57]:
# deklaracja klasy dziedziczącej po Model
class Person(Model):
    id = Field(FieldType.INTEGER)
    firstname = Field(FieldType.STRING)
    lastname = Field(FieldType.STRING)
    age = Field(FieldType.INTEGER)

In [31]:
p = Person()
p.db_table

'db_person'

In [58]:
p.id

In [32]:
p.get_fields()

{'age': 'Field', 'firstname': 'Field', 'id': 'Field', 'lastname': 'Field'}

In [19]:
test = Model.from_dict('Movie',{'id': Field(FieldType.INTEGER), 'name': Field(FieldType.STRING)})

In [20]:
test.get_fields()

{'id': 'Field', 'name': 'Field'}

In [21]:
test.db_table

'db_movie'

## 2. Abstrakcyjne klasy bazowe (ABC - Abstract Base Classes)

Abstrakcyjne klasy bazowe Pythona zostały zdefiniowane w module abc i zawierają zbiór klas, które są często powiązane ze soba poprzez dziedziczenie dostarczając coraz to większej ilości metod abstrakcyjnych, które konkretna klasa musi zapewnić.

> Listę tych klas oraz dziedziczenia znajdziesz tu: https://docs.python.org/3/library/collections.abc.html

Warto w tym kontekście poznać termin kacze typowanie (ang. duck typing): https://devopedia.org/duck-typing

**Listing 2**

In [8]:
# a teraz przykład z Field jako klasa abstrakcyjna
from abc import ABC, abstractmethod


class Field(ABC):

    def __init__(self):
        self.value = None

    def get_fieldtype(self):
        return self.__class__.__name__

    def __setatrr__(self, attr, val):
        if attr == 'value':
            self._set_field_value(val)
        else:
            super().__setattr__(self,attr,val)

    @abstractmethod
    def _get_field_value(self):
        ...

    @abstractmethod
    def _set_field_value(self, val):
        ...

    def __str__(self):
        return self.__class__.__name__


class StringField(Field):

    def _set_field_value(self, val):
        if isinstance(val, str):
            self.value = val

    def _get_field_value(self):
        return self.value


In [42]:
# próba inicjalizacji klasy abstrakcyjnej
field = Field()

TypeError: Can't instantiate abstract class Field without an implementation for abstract methods '_get_field_value', '_set_field_value'

In [9]:
# kod w klasie Model pozwala na przypisanie właściwości typu Field do instancji klasy model
class Movie(Model):
    title = StringField()
    director = StringField()

In [11]:
# do samego pola klasy model dzięki odpowiedniej inmplementacji przypisywana jest wartość
# pozostawiając wciąż pole klasą konkretną dziedziczącą po Field
movie = Movie()
print(movie.title)
movie.title = 'Pierwszy człowiek'
print(movie.title)

StringField
StringField


In [45]:
movie.title.get_fieldtype()

'StringField'

In [46]:
movie.db_table

'db_movie'

In [14]:
movie.title

**Zadania**

> **UWAGA!**
> 
> Zachowaj oryginalny kod źródłowy przedstawiony w przykładach, a swoje rozwiązanie implementuj w nowych komórkach kodu.

1. Dokonaj refaktoryzacji klasy `Model` i zamień logikę w inicjalizatorze tak, aby ustawianie wartości atrybutu `db_table` odbywało się poprzez wywołanie odpowiedniej metody (jest aktualnie zdefiniowana jako statyczna).

2. Bazując na listingu 1 sprawdź czy możliwe jest zadeklarowanie klasy Enum `FieldType` jako klasy wewnętrznej klasy `Field` i przetestuj działanie takiej implementacji.

3. Dokonaj refaktoryzacji kodu klas `Field` oraz klasy `StringField` zmieniając deklarację metod `_get_field_value` oraz `_set_field_value` tak aby były ukryte (według konwencji Pythona). Przetestuj czy rozwiązanie działa poprawnie przy przypisywaniu wartości dla tej klasy.

4. Dopisanie logiki w klasach `Field` (bazowej lub dziedziczących - zdecyduj), aby po wywołaniu np. w przykładach `movie.title` wyświetlały wartość tego pola modelu, a nie `<__main__.StringField at ...>`.

5. Bazując na listingu 2 zaimpementuj dwie klasy implementujące `Field` - `IntegerField` oraz `DateField`, które powinny działac w podobny sposób. Sprawdzamy czy wartość im przypisywana jest określonego typu (tu nie musi to być doskonałe rozwiązanie).

6. Dopisz do klasy `Model` metodę `save`, która będzie zwracała poprawny (plus/minus) SQL, który wstawi dane do tabeli o nazwie odpowiadającej zmiennej `db_table` z zachowaniem poniższych warunków:

* wykonujemy `INSERT` jeżeli `id` nie ma przypisanej wartości i wtedy dla tej kolumny dajemy wartość `default`,
* wykonujemy `UPDATE` jeżeli `id` ma przypisaną wartość,
* pozostałe pola (instancje) klasy `Field` danego modelu również znajdują się w tym 'zapytaniu'.

7. Sprawdź w dokumentacji (link w punkcie 2) jak wygląda dziedziczenie w schemacie klas ABC i zaimplementuj dwie klasy:

* klasę `Koszyk` dziedziczącą po klasie `MutableSequence` - zaimplementuj niezbędne metody
* klasę `Tydzien` z dodanymi w kodzie nazwami dni tygodnia dziedziczącą po klasie `Collection`
* dodaj kod testujący działanie wszystkich zaimplementowanych metod
